# **Interpretation: Zonal Statistics Analysis of MODIS Datasets**

This study compares district-level vegetation and land surface temperature patterns across India using three MODIS-derived variables: Percent Tree Cover (MOD44B), Leaf Area Index (MCD15A3H), and Land Surface Temperature (MOD11A1). Two time periods were analyzed: the early 2000s (2001–2003) and the recent period (2024–2025). District-wise zonal statistics were computed to examine spatial distribution patterns and identify environmental changes over time.

In [ ]:
import ee

In [ ]:
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='tanu-p-483509')

In [ ]:
import geemap as map
map

<module 'geemap' from '/usr/local/lib/python3.12/dist-packages/geemap/__init__.py'>

# **Spatial Patterns: Early 2000s (2001–2003)**

In the early 2000s, high tree cover and LAI values were concentrated in the northeastern states, the Western Ghats, and Himalayan districts. These regions exhibited dense forest cover and high vegetation productivity, reflected in elevated LAI values. Districts in Arunachal Pradesh, Assam, Kerala, and parts of Uttarakhand showed strong vegetation presence.

Conversely, western India, particularly Rajasthan and parts of Gujarat, displayed low tree cover and low LAI values, indicating arid and semi-arid climatic conditions. Central plateau regions exhibited moderate vegetation levels.

Land Surface Temperature patterns during this period showed higher temperatures in northwestern and central India, where vegetation cover was sparse. Lower temperatures were observed in forested and high-altitude districts. A clear inverse relationship between vegetation density and temperature was visible, with greener districts exhibiting comparatively cooler surface temperatures.

In [ ]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
Districts = ee.FeatureCollection("projects/tanu-p-483509/assets/District_Boundary")
bounds_geometry = Districts.bounds()
center_point_geometry = bounds_geometry.centroid(ee.ErrorMargin(1, 'meters'))
map.addLayer(Districts, {}, 'Districts')
map.centerObject(center_point_geometry, 5)
map

Map(bottom=812.0, center=[21.589225894657407, 82.79520439021228], controls=(WidgetControl(options=['position',…

In [ ]:
start1 = '2001-01-01'
end1   = '2003-12-31'
start2 = '2024-01-01'
end2   = '2025-12-31'

In [ ]:
MOD44B = ee.ImageCollection("MODIS/061/MOD44B").select("Percent_Tree_Cover")
Tree_early = MOD44B.filterDate(start1, end1).mean()
Tree_recent = MOD44B.filterDate(start2, end2).mean()
vis_tree = {
    'min': 0,
    'max': 100,
    'palette': ['yellow', 'green', 'darkgreen']
}
map.addLayer(Tree_early.clip(Districts), vis_tree, "Tree Cover 2001-03")
map.addLayer(Tree_recent.clip(Districts), vis_tree, "Tree Cover 2024-25")
map


Map(bottom=3893.0, center=[21.589225894657407, 82.79520439021228], controls=(WidgetControl(options=['position'…

In [ ]:
LAI_collection = ee.ImageCollection("MODIS/061/MCD15A3H").select("Lai")
LAI_early = LAI_collection.filterDate(start1, end1).mean()
LAI_recent = LAI_collection.filterDate(start2, end2).mean()
LAI_early = LAI_early.multiply(0.1)
LAI_recent = LAI_recent.multiply(0.1)
vis_lai = {
    'min': 0,
    'max': 6,
    'palette': ['brown', 'yellow', 'green']
}
map.addLayer(LAI_early.clip(Districts), vis_lai, "LAI 2001-03")
map.addLayer(LAI_recent.clip(Districts), vis_lai, "LAI 2024-25")
map

Map(bottom=3809.0, center=[24.966140159912975, 80.04182291373952], controls=(WidgetControl(options=['position'…

In [ ]:
LST_collection = ee.ImageCollection("MODIS/061/MOD11A1").select("LST_Day_1km")
LST_early = LST_collection.filterDate(start1, end1).mean()
LST_recent = LST_collection.filterDate(start2, end2).mean()
LST_early = LST_early.multiply(0.02).subtract(273.15)
LST_recent = LST_recent.multiply(0.02).subtract(273.15)
vis_lst = {
    'min': 20,
    'max': 45,
    'palette': ['blue', 'yellow', 'red']
}
map.addLayer(LST_early.clip(Districts), vis_lst, "LST 2001-03")
map.addLayer(LST_recent.clip(Districts), vis_lst, "LST 2024-25")
map

Map(bottom=3729.0, center=[28.11074876063356, 78.46563122980481], controls=(WidgetControl(options=['position',…

# **Spatial Patterns: Recent Period (2024–2025)**

In the recent period, vegetation patterns show both continuity and change. Forest-rich regions in the Northeast and Western Ghats continue to display high tree cover; however, some districts show slight reductions in tree cover values, possibly indicating localized deforestation or land-use conversion.

Certain agricultural districts, particularly in Punjab, Haryana, and parts of central India, demonstrate stable or slightly increased LAI values, which may be associated with intensified irrigation and multi-cropping systems.

Land Surface Temperature in 2024–2025 appears higher across many districts compared to the early 2000s. Urban districts and rapidly expanding metropolitan regions show noticeably elevated temperature values. The increase in LST is especially prominent in densely populated and industrializing areas.

In [ ]:
Tree_early_stats = Tree_early.reduceRegions(
    collection=Districts,
    reducer=ee.Reducer.mean(),
    scale=500
)
Tree_recent_stats = Tree_recent.reduceRegions(
    collection=Districts,
    reducer=ee.Reducer.mean(),
    scale=500
)

In [ ]:
LAI_early_stats = LAI_early.reduceRegions(
    collection=Districts,
    reducer=ee.Reducer.mean(),
    scale=500
)
LAI_recent_stats = LAI_recent.reduceRegions(
    collection=Districts,
    reducer=ee.Reducer.mean(),
    scale=500
)

In [ ]:
LST_early_stats = LST_early.reduceRegions(
    collection=Districts,
    reducer=ee.Reducer.mean(),
    scale=1000
)
LST_recent_stats = LST_recent.reduceRegions(
    collection=Districts,
    reducer=ee.Reducer.mean(),
    scale=1000
)

In [ ]:
geemap.ee_export_vector_to_drive(Tree_early_stats, "Tree_2001_03.geojson", folder='earthengine_exports', fileFormat='geojson')
geemap.ee_export_vector_to_drive(Tree_recent_stats, "Tree_2024_25.geojson", folder='earthengine_exports', fileFormat='geojson')
geemap.ee_export_vector_to_drive(LAI_early_stats, "LAI_2001_03.geojson", folder='earthengine_exports', fileFormat='geojson')
geemap.ee_export_vector_to_drive(LAI_recent_stats, "LAI_2024_25.geojson", folder='earthengine_exports', fileFormat='geojson')
geemap.ee_export_vector_to_drive(LST_early_stats, "LST_2001_03.geojson", folder='earthengine_exports', fileFormat='geojson')
geemap.ee_export_vector_to_drive(LST_recent_stats, "LST_2024_25.geojson", folder='earthengine_exports', fileFormat='geojson')

Exporting Tree_2001_03.geojson... Please check the Task Manager from the JavaScript Code Editor.
Exporting Tree_2024_25.geojson... Please check the Task Manager from the JavaScript Code Editor.
Exporting LAI_2001_03.geojson... Please check the Task Manager from the JavaScript Code Editor.
Exporting LAI_2024_25.geojson... Please check the Task Manager from the JavaScript Code Editor.
Exporting LST_2001_03.geojson... Please check the Task Manager from the JavaScript Code Editor.
Exporting LST_2024_25.geojson... Please check the Task Manager from the JavaScript Code Editor.


In [ ]:
Tree_change = Tree_recent.subtract(Tree_early)
LAI_change = LAI_recent.subtract(LAI_early)
LST_change = LST_recent.subtract(LST_early)
vis_change = {
    'min': -10,
    'max': 10,
    'palette': ['red', 'white', 'green']
}
map.addLayer(Tree_change.clip(Districts), vis_change, "Tree Change")
map.addLayer(LAI_change.clip(Districts), vis_change, "LAI Change")
map.addLayer(LST_change.clip(Districts), vis_change, "LST Change")
map

Map(bottom=3729.0, center=[28.11074876063356, 78.46563122980481], controls=(WidgetControl(options=['position',…

# **Comparative Analysis and Observed Changes**

Comparing the two periods reveals several significant trends. Many urbanizing districts exhibit a reduction in tree cover alongside an increase in land surface temperature. This pattern is consistent with urban expansion and the replacement of vegetation by built-up surfaces.

Some central Indian districts show moderate declines in vegetation, which may be linked to mining activity, infrastructure development, or land conversion. At the same time, a few agricultural regions show stable or increased LAI values due to irrigation expansion and improved agricultural productivity.

The increase in land surface temperature across multiple districts suggests both localized urban heat island effects and broader climatic warming trends. Districts with noticeable vegetation decline often correspond with stronger increases in surface temperature, reinforcing the ecological relationship between vegetation cover and surface cooling through evapotranspiration.

In [ ]:
from google.colab import files
map.save("Zonal_Statistics_MODIS.html")
files.download("Zonal_Statistics_MODIS.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def add_centroid(feature):
    centroid = feature.geometry().centroid()
    coords = centroid.coordinates()
    return feature.set({
        'longitude': coords.get(0),
        'latitude': coords.get(1)
    })

District_with_coords = Districts.map(add_centroid)

# **Possible Causes of Observed Changes**

The observed changes may be attributed to several interacting factors:

Rapid urbanization and infrastructure development
• Expansion of industrial corridors
• Deforestation and land-use change
• Agricultural intensification and irrigation expansion
• Climate variability and long-term warming trends

Urban heat island effects are particularly visible in metropolitan regions where vegetation loss corresponds with higher surface temperatures. Conversely, irrigated agricultural belts show relatively stable vegetation indices despite broader climatic changes.

In [ ]:
import os

In [ ]:

output_folder = output_folder = "/content"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
Tree_early_csv = os.path.join(output_folder, "Tree_2001_03_STATS.csv")
geemap.zonal_stats(
    Tree_early,
    District_with_coords,
    Tree_early_csv,
    stat_type="MEAN",
    scale=500
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/Tree_2001_03_STATS.csv


In [ ]:
Tree_recent_csv = os.path.join(output_folder, "Tree_2024_25_STATS.csv")
geemap.zonal_stats(
    Tree_early,
    District_with_coords,
    Tree_early_csv,
    stat_type="MEAN",
    scale=500
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/Tree_2001_03_STATS.csv


In [ ]:
LAI_early_csv = os.path.join(output_folder, "LAI_2001_03_STATS.csv")
geemap.zonal_stats(
    Tree_early,
    District_with_coords,
    Tree_early_csv,
    stat_type="MEAN",
    scale=500
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/Tree_2001_03_STATS.csv


In [ ]:
LAI_recent_csv = os.path.join(output_folder, "LAI_2024_25_STATS.csv")
geemap.zonal_stats(
    LAI_recent,
    Districts,
    LAI_recent_csv,
    stat_type="MEAN",
    scale=500
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/LAI_2024_25_STATS.csv


In [ ]:
LST_early_csv = os.path.join(output_folder, "LST_2001_03_STATS.csv")
geemap.zonal_stats(
    LST_early,
    Districts,
    LST_early_csv,
    stat_type="MEAN",
    scale=1000
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/LST_2001_03_STATS.csv


In [ ]:
LST_recent_csv = os.path.join(output_folder, "LST_2024_25_STATS.csv")
geemap.zonal_stats(
    LST_recent,
    Districts,
    LST_recent_csv,
    stat_type="MEAN",
    scale=1000
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/LST_2024_25_STATS.csv


# **Limitations and Future Scope**

This analysis is subject to certain limitations. MODIS datasets have moderate spatial resolution (250 m to 1 km), which may not capture fine-scale heterogeneity within districts. Aggregating values at the district level may mask intra-district variations. Additionally, only two time periods were compared, limiting the ability to detect long-term trends.

Future studies could incorporate multi-year trend analysis, higher-resolution satellite datasets, seasonal comparisons, and integration with socio-economic or land-use datasets. Advanced techniques such as district-level trend modeling or clustering analysis could provide deeper insights into environmental transitions across India.

GitHub Link : https://github.com/tanu-pawar/Zonal-Statistics-MODIS-India